<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/HubSpot_Logo.svg/220px-HubSpot_Logo.svg.png" alt="drawing" width="200" align='left'/>

# Hubspot - Send sales brief
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Qonto/Qonto_Releve_de_compte_augmente.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

#hubspot #crm #sales #emailbuilder

## Input

### Import library

In [1]:
from naas_drivers import emailbuilder
import naas
import pandas as pd
from datetime import datetime
from hubspot import Hubspot
hubspot = Hubspot()

### Enter your Hubspot api key

In [2]:
auth_token = 'YOUR_HUBSPOT_API_KEY'

In [3]:
auth_token = naas.secret.get("HUBSPOT_API_KEY")

### Connect to Hubspot

In [4]:
hs = hubspot.connect(auth_token)

### Email parameters

In [5]:
# Receivers
email_to = ["florent.ravenel@cashstory.com"]

# Email subject
email_subject = f"🚀 Hubspot - Sales Brief as of {datetime.now().strftime('%d/%m/%Y')}"

### Schedule automation

In [6]:
# naas.scheduler.add(cron="0 8 * * 1")

### Pick your pipeline

#### Get all pipelines

In [7]:
df_pipelines = hs.pipelines.get_all()
df_pipelines

,pipeline,pipeline_id,dealstage_label,dealstage_id,displayOrder,dealclosed,probability,createdAt,updatedAt,archived
0,Archive,default,Activity 2020,9234636,0,true,1.0,1970-01-01 00:00:00,2021-06-06 20:56:29,False
1,Archive,default,Activity 2019,closedwon,1,true,1.0,2017-02-26 11:33:34,2021-06-06 20:56:29,False
2,Archive,default,Activity 2018,42d9fab1-9386-49c2-a8fb-53ceb3f94888,2,true,1.0,1970-01-01 00:00:00,2021-06-06 20:56:29,False
3,Archive,default,Activity 2017,presentationscheduled,3,true,1.0,2017-02-26 11:33:34,2021-06-06 20:56:29,False
4,Archive,default,Lost deals,39525fcb-3e49-4aab-873d-909393447678,4,true,0.0,2017-05-24 08:23:11,2021-06-06 20:56:29,False
5,Onboarding,13412431,Register / Credentials,13412432,0,false,0.2,2021-04-19 10:17:00,2021-06-06 20:56:28,False
6,Onboarding,13412431,Nice to meet you,13412433,1,false,0.4,2021-04-19 10:17:00,2021-06-06 20:56:28,False
7,Onboarding,13412431,Product,13412434,2,false,0.6,2021-04-19 10:17:00,2021-06-06 20:56:28,False
8,Onboarding,13412431,Use Case,13412435,3,false,0.8,2021-04-19 10:17:00,2021-06-06 20:56:28,False
9,Onboarding,13412431,OS Community,13412436,4,false,0.9,2021-04-19 10:17:00,2021-06-06 20:56:28,False


#### Enter your pipeline id

In [8]:
pipeline_id = "8432671"

### Get dealstages from pipeline

In [9]:
df_dealstages = df_pipelines.copy()

# Filter on pipeline
df_dealstages = df_dealstages[df_dealstages.pipeline_id == pipeline_id]

df_dealstages

,pipeline,pipeline_id,dealstage_label,dealstage_id,displayOrder,dealclosed,probability,createdAt,updatedAt,archived
12,Sales pipeline,8432671,Lead,8432672,0,false,0.1,2020-12-16 14:50:35,2021-07-21 08:27:28,False
13,Sales pipeline,8432671,Analyse,8432673,1,false,0.3,2020-12-16 14:50:35,2021-07-21 08:27:28,False
14,Sales pipeline,8432671,Negociation,8432674,2,false,0.6,2020-12-16 14:50:35,2021-07-21 08:27:28,False
15,Sales pipeline,8432671,Closing,8432675,3,false,0.8,2020-12-16 14:50:35,2021-07-21 08:27:28,False
16,Sales pipeline,8432671,Invoiced,8432676,4,true,1.0,2020-12-16 14:50:35,2021-07-21 08:27:28,False
17,Sales pipeline,8432671,Cash-in,9322543,5,true,1.0,1970-01-01 00:00:00,2021-07-21 08:27:28,False
18,Sales pipeline,8432671,Lost,8432678,6,true,0.0,2020-12-16 14:50:35,2021-07-21 08:27:28,False


### Get deals from pipeline

In [10]:
properties = [
    "hs_object_id",
    "dealname",
    "dealstage",
    "pipeline",
    "createdate",
    "closedate",
    "amount"
]
df_deals = hs.deals.get_all(properties)

# Filter on pipeline
df_deals = df_deals[df_deals.pipeline == pipeline_id].reset_index(drop=True)

df_deals

,hs_object_id,dealname,dealstage,pipeline,createdate,closedate,amount
0,306763502,Nexity - Finance,8432672,8432671,2018-06-06T17:57:46.338Z,2018-06-30T17:57:46.338Z,
1,1147611903,ARKEMA,8432672,8432671,2019-11-06T09:47:50.022Z,2019-11-30T09:47:50.015Z,None
2,1495383475,Groupama - COMEX finance,8432672,8432671,2020-01-30T09:30:09.976Z,2020-01-31T09:30:09.975Z,12000
3,1513502857,ETAM - Carole Chegefti,8432672,8432671,2020-02-04T16:51:36.699Z,2020-02-29T16:51:36.699Z,None
4,1804146591,Kering - HK,8432672,8432671,2020-03-30T13:33:38.889Z,2020-03-31T13:33:38.887Z,None
...,...,...,...,...,...,...,...
164,5646881615,Airseas : Automation FashReport,8432673,8432671,2021-07-09T08:57:33.950Z,2021-07-31T08:57:33.950Z,10000
165,5681366937,Bouygues - Formation Trésorerie,8432672,8432671,2021-07-14T08:43:59.128Z,2021-07-31T08:43:59.128Z,None
166,5718648953,L'Oréal - Sujet Cash Forecasting Anaplan,8432672,8432671,2021-07-19T13:04:49.657Z,2021-07-31T13:04:49.657Z,None
167,5718730922,Maxime Blasco - LinkedIn setup,8432672,8432671,2021-07-19T13:23:49.388Z,2021-07-31T13:23:49.388Z,220


## Model

### Calculate yearly forecasted

In [11]:
df_sales = pd.merge(df_deals.drop("pipeline", axis=1),
                    df_dealstages.drop(["pipeline", "pipeline_id", "createdAt", "updatedAt", "archived"], axis=1),
                    left_on="dealstage",
                    right_on="dealstage_id",
                    how="left")
df_sales

,hs_object_id,dealname,dealstage,createdate,closedate,amount,dealstage_label,dealstage_id,displayOrder,dealclosed,probability
0,306763502,Nexity - Finance,8432672,2018-06-06T17:57:46.338Z,2018-06-30T17:57:46.338Z,,Lead,8432672,0,false,0.1
1,1147611903,ARKEMA,8432672,2019-11-06T09:47:50.022Z,2019-11-30T09:47:50.015Z,None,Lead,8432672,0,false,0.1
2,1495383475,Groupama - COMEX finance,8432672,2020-01-30T09:30:09.976Z,2020-01-31T09:30:09.975Z,12000,Lead,8432672,0,false,0.1
3,1513502857,ETAM - Carole Chegefti,8432672,2020-02-04T16:51:36.699Z,2020-02-29T16:51:36.699Z,None,Lead,8432672,0,false,0.1
4,1804146591,Kering - HK,8432672,2020-03-30T13:33:38.889Z,2020-03-31T13:33:38.887Z,None,Lead,8432672,0,false,0.1
...,...,...,...,...,...,...,...,...,...,...,...
164,5646881615,Airseas : Automation FashReport,8432673,2021-07-09T08:57:33.950Z,2021-07-31T08:57:33.950Z,10000,Analyse,8432673,1,false,0.3
165,5681366937,Bouygues - Formation Trésorerie,8432672,2021-07-14T08:43:59.128Z,2021-07-31T08:43:59.128Z,None,Lead,8432672,0,false,0.1
166,5718648953,L'Oréal - Sujet Cash Forecasting Anaplan,8432672,2021-07-19T13:04:49.657Z,2021-07-31T13:04:49.657Z,None,Lead,8432672,0,false,0.1
167,5718730922,Maxime Blasco - LinkedIn setup,8432672,2021-07-19T13:23:49.388Z,2021-07-31T13:23:49.388Z,220,Lead,8432672,0,false,0.1


In [39]:
df_sales_c = df_sales.copy()

# Cleaning
df_sales_c["amount"] = df_sales_c["amount"].fillna("0")
df_sales_c.loc[df_sales_c["amount"] == "", "amount"] = "0"

# Formatting
df_sales_c["amount"] = df_sales_c["amount"].astype(float)
df_sales_c["probability"] =  df_sales_c["probability"].astype(float)

# Calc
df_sales_c["forecasted"] = df_sales_c["amount"] * df_sales_c["probability"]

df_sales_c

,hs_object_id,dealname,dealstage,createdate,closedate,amount,dealstage_label,dealstage_id,displayOrder,dealclosed,probability,forecasted
0,306763502,Nexity - Finance,8432672,2018-06-06T17:57:46.338Z,2018-06-30T17:57:46.338Z,0.0,Lead,8432672,0,false,0.1,0.0
1,1147611903,ARKEMA,8432672,2019-11-06T09:47:50.022Z,2019-11-30T09:47:50.015Z,0.0,Lead,8432672,0,false,0.1,0.0
2,1495383475,Groupama - COMEX finance,8432672,2020-01-30T09:30:09.976Z,2020-01-31T09:30:09.975Z,12000.0,Lead,8432672,0,false,0.1,1200.0
3,1513502857,ETAM - Carole Chegefti,8432672,2020-02-04T16:51:36.699Z,2020-02-29T16:51:36.699Z,0.0,Lead,8432672,0,false,0.1,0.0
4,1804146591,Kering - HK,8432672,2020-03-30T13:33:38.889Z,2020-03-31T13:33:38.887Z,0.0,Lead,8432672,0,false,0.1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
164,5646881615,Airseas : Automation FashReport,8432673,2021-07-09T08:57:33.950Z,2021-07-31T08:57:33.950Z,10000.0,Analyse,8432673,1,false,0.3,3000.0
165,5681366937,Bouygues - Formation Trésorerie,8432672,2021-07-14T08:43:59.128Z,2021-07-31T08:43:59.128Z,0.0,Lead,8432672,0,false,0.1,0.0
166,5718648953,L'Oréal - Sujet Cash Forecasting Anaplan,8432672,2021-07-19T13:04:49.657Z,2021-07-31T13:04:49.657Z,0.0,Lead,8432672,0,false,0.1,0.0
167,5718730922,Maxime Blasco - LinkedIn setup,8432672,2021-07-19T13:23:49.388Z,2021-07-31T13:23:49.388Z,220.0,Lead,8432672,0,false,0.1,22.0


In [54]:
df_details = df_sales_c.copy()

# Groupby
to_group = [
    "dealstage_label",
    "probability",
    "displayOrder"
]
to_agg = {
    "amount": "sum",
    "dealname": "count",
    "forecasted": "sum"
}
df_details = df_details.groupby(to_group, as_index=False).agg(to_agg)

# # Format
# df_details["amount"] = df_details["amount"].map(NUMBER_FORMAT.format).str.replace(",", " ")
# df_details["forecasted"] = df_details["forecasted"].map(NUMBER_FORMAT.format).str.replace(",", " ")

# Sort
df_details = df_details.sort_values("displayOrder")

df_details

,dealstage_label,probability,displayOrder,amount,dealname,forecasted
4,Lead,0.1,0,213604.00,80,21360.40
0,Analyse,0.3,1,99636.00,14,29890.80
6,Negociation,0.6,2,34664.00,3,20798.40
2,Closing,0.8,3,15100.00,3,12080.00
3,Invoiced,1.0,4,10286.20,5,10286.20
1,Cash-in,1.0,5,62180.55,27,62180.55
5,Lost,0.0,6,37165.00,37,0.00


In [187]:
forecasted = df_details.forecasted.sum()
forecasted

156596.34999999998

In [188]:
won = df_details[df_details["dealstage_label"] == "Cash-in"].forecasted.sum()
won

62180.55

In [189]:
weighted = df_details[df_details["dealstage_label"] != "Cash-in"].forecasted.sum()
weighted

94415.79999999999

In [190]:
objective = 300000
objective

300000

In [191]:
completion_p = forecasted / objective
completion_p

0.5219878333333332

In [192]:
completion_v = objective - forecasted
completion_v

143403.65000000002

In [193]:
DATE_FORMAT = "%Y-%m-%d"

today = datetime.now().strftime(DATE_FORMAT)
today

'2021-08-03'

In [194]:
df = df_details.copy()

details = []

for _, row in df.iterrows():
    # status part
    dealstage = row.dealstage_label
    print(dealstage)
    probability = row.probability
    detail = f"{dealstage} ({format_pourcentage(probability)})"
    
    # amount part
    amount = row.amount
    number = row.dealname
    forecasted_ = row.forecasted
    if (probability < 1 and probability > 0):
        detail = f"{detail}: <ul><li>Amount : {format_number(amount)}</li><li>Number : {number}</li><li>Weighted amount : {format_number(forecasted_)}</li></ul>"
    else:
        detail = f"{detail}: {format_number(amount)}"
        
    details += [detail]

details

Lead
Analyse
Negociation
Closing
Invoiced
Cash-in
Lost


['Lead (10%): <ul><li>Amount : 213 604 €</li><li>Number : 80</li><li>Weighted amount : 21 360 €</li></ul>',
 'Analyse (30%): <ul><li>Amount : 99 636 €</li><li>Number : 14</li><li>Weighted amount : 29 891 €</li></ul>',
 'Negociation (60%): <ul><li>Amount : 34 664 €</li><li>Number : 3</li><li>Weighted amount : 20 798 €</li></ul>',
 'Closing (80%): <ul><li>Amount : 15 100 €</li><li>Number : 3</li><li>Weighted amount : 12 080 €</li></ul>',
 'Invoiced (100%): 10 286 €',
 'Cash-in (100%): 62 181 €',
 'Lost (0%): 37 165 €']

In [195]:
def format_number(num):
    NUMBER_FORMAT = "{:,.0f} €"
    num = str(NUMBER_FORMAT.format(num)).replace(",", " ")
    return num

In [196]:
def format_pourcentage(num):
    NUMBER_FORMAT = "{:,.0%}"
    num = str(NUMBER_FORMAT.format(num))
    return num

## Output

### Create email

In [197]:
def email_brief(today,
                forecasted,
                won,
                weighted,
                objective,
                completion_p,
                completion_v,
                details,
                ):
    content = {
        'txt_intro': (f"Hi there,<br><br>"
                      f"Here is your weekly sales email as of {today}."),

        'title_1': emailbuilder.text("Overview", font_size="27px", text_align="center", bold=True, underline=True),
        "text_1": emailbuilder.text(f"As of today, your yearly forecasted revenue is <b>{format_number(forecasted)}</b>."),
        "list_1": emailbuilder.list([f"Won : {format_number(won)}",
                                     f"Weighted pipeline : {format_number(weighted)}"]),
        "text_1_1": emailbuilder.text(f"Your yearly objective is {format_number(objective)} ({format_pourcentage(completion_p)} completion)."),
        "text_1_2": emailbuilder.text(f"You need to find <b>{format_number(completion_v)}</b> to reach your goal !"),
        
        'title_2': emailbuilder.text("Sales pipeline", font_size="27px", text_align="center", bold=True, underline=True),
        "list_2": emailbuilder.list(details),

        'title_3': emailbuilder.text("Inactive deals", font_size="27px", text_align="center", bold=True, underline=True),
        'text_3': emailbuilder.text("Here are deals where you need to take actions"),
        'list_3': emailbuilder.list(["YSL - Projet migration PowerBI : 25 588 €", "...", "..."]),
        'text_3_1': emailbuilder.text("If you need more details, jump into <a href='https://app.hubspot.com/'>Hubspot</a>."),
        
        'title_4': emailbuilder.text("Glossary", bold=True, underline=True),
        'list_4': emailbuilder.list(["Yearly forecasted revenue :  Weighted amount + WON exclude LOST",
                                     "Yearly objective : Input in script",
                                     "Inactive deal : No activity for more than 10 days"]),
        
        'footer_cs': emailbuilder.footer_company(naas=True),
    }
    
    email_content = emailbuilder.generate(display='iframe', **content)
    return email_content

email_content = email_brief(today,
                            forecasted,
                            won,
                            weighted,
                            objective,
                            completion_p,
                            completion_v,
                            details)

### Send email

In [ ]:
naas.notification.send(email_to,
                       email_subject,
                       email_content)